<a href="https://colab.research.google.com/github/Afrah333333/AFRAH/blob/main/FASTAPI_work_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn python-multipart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=4753a901ac5eff5ffbf761f9aceb848f0cc00dfdbf3733528282d0ea88ffc913
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok


In [2]:
from google.colab import files
upload = files.upload()

uploads = files.upload()

uploadss = files.upload()

Saving my_model (1).h5 to my_model (1).h5


Saving model_weights.h5 to model_weights.h5


Saving model_architecture.json to model_architecture.json


In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model

import tensorflow as tf

# Load the model architecture from the JSON file
with open('/content/model_architecture.json', 'r') as json_file:
    model_json = json_file.read()
MODEL = tf.keras.models.model_from_json(model_json)

# Load the model weights
MODEL.load_weights('model_weights.h5')

# Now, you have the loaded_model ready for prediction





In [7]:
from re import U
from fastapi import FastAPI,File,UploadFile
import nest_asyncio
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import numpy as np
from io import BytesIO
from PIL import Image
import tensorflow as tf
import requests

app = FastAPI()



CLASS_NAMES = ["Early Blight", "Late Blight", "Healthy"]
print('hi')


def read_file_as_image(data) -> np.ndarray:
    image = np.array(Image.open(BytesIO(data)))
    return image

@app.post("/predict")
async def predict(
    file: UploadFile = File(...)
):
    # Log the incoming request and route
    print(f"Incoming request to /predict, method: {requests.method}")

    image = read_file_as_image(await file.read())
    img_batch = np.expand_dims(image, 0)

    predictions = MODEL.predict(img_batch)

    predicted_class = CLASS_NAMES[np.argmax(predictions[0])]
    confidence = np.max(predictions[0])
    return {
        'class': predicted_class,
        'confidence': float(confidence)
    }
from fastapi import FastAPI, Request


print(f"Incoming request to /predict, method: {Request.method}")

ngrok_tunnel = ngrok.connect(8054)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8054)


hi
Incoming request to /predict, method: <property object at 0x7a830d50e9d0>
Public URL: https://69e8-34-147-38-164.ngrok.io


INFO:     Started server process [276]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8054 (Press CTRL+C to quit)


INFO:     102.222.234.131:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     102.222.234.131:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     102.222.232.196:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     102.222.234.131:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     102.222.234.131:0 - "GET /predict HTTP/1.1" 405 Method Not Allowed
INFO:     102.222.234.131:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     102.222.232.197:0 - "GET /docs HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [276]


In [5]:
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import numpy as np
from io import BytesIO
from PIL import Image
import tensorflow as tf
import requests

app = FastAPI()

origins = [
    "http://localhost",
    "http://localhost:3000",
]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

endpoint = "http://localhost:8501/v1/models/potatoes_model:predict"

CLASS_NAMES = ["Early Blight", "Late Blight", "Healthy"]

@app.get("/ping")
async def ping():
    return "Hello, I am alive"

def read_file_as_image(data) -> np.ndarray:
    image = np.array(Image.open(BytesIO(data)))
    return image

@app.post("/predict")
async def predict(
    file: UploadFile = File(...)
):
    image = read_file_as_image(await file.read())
    img_batch = np.expand_dims(image, 0)

    json_data = {
        "instances": img_batch.tolist()
    }

    response = requests.post(endpoint, json=json_data)
    prediction = np.array(response.json()["predictions"][0])

    predicted_class = CLASS_NAMES[np.argmax(prediction)]
    confidence = np.max(prediction)

    return {
        "class": predicted_class,
        "confidence": float(confidence)
    }


ngrok_tunnel = ngrok.connect(8082)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8082)

Public URL: https://1405-34-147-38-164.ngrok.io


INFO:     Started server process [276]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8082 (Press CTRL+C to quit)


INFO:     102.222.234.131:0 - "GET / HTTP/1.1" 405 Method Not Allowed
INFO:     102.222.234.131:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [276]
